In [17]:
import pandas as pd
import pandasql
import os


os.chdir("/Users/bencampbell/code_louisville/capstone/louisville-bike-accidents/")

DATA = "data/clean/bike_accidents.csv"
assert os.path.exists(DATA)

DATA = pd.read_csv(DATA) 

SIGDATA = "data/preclean/signalized_intersections.csv"

SIG = pd.read_csv(SIGDATA)

# Set hash as index for testing
#DATA['hash'] = pd.util.hash_pandas_object(DATA)
#SIG['hash'] = pd.util.hash_pandas_object(SIG)

def query(query, **names):
    if not names:
        names = {"DATA":DATA, "SIG":SIG}
    return pandasql.sqldf(query, names)


In [18]:
# Rearrange roadway name/number intersection name/number cols
DATA['main_roadway'] = DATA['roadway_name'].combine_first(DATA['roadway_number'])
DATA['intersection_roadway'] = DATA['intersection_roadway_name'].combine_first(DATA['intersection_roadway_number'])
DATA['intersection_indicator'] = DATA['intersection_roadway'].notnull()



In [19]:
#          SELECT DATA.main_roadway, DATA.intersection_roadway, SIG.main_street, SIG.cross_street FROM

a = pandasql.sqldf("""SELECT DATA.main_roadway, DATA.intersection_roadway FROM
      DATA JOIN SIG 
      ON DATA.main_roadway == SIG.main_street 
      AND DATA.intersection_roadway == SIG.cross_street""")




In [20]:
a = pandasql.sqldf("""SELECT DATA.id FROM
      DATA INNER JOIN SIG 
      ON DATA.main_roadway == SIG.main_street 
      AND DATA.intersection_roadway == SIG.cross_street""")
v = a.id.to_list()
DATA[DATA.id.isin(v)]

,investigating_agency,roadway_number,building_number,roadway_name,roadway_suffix,roadway_direction,milepoint,intersection_roadway_number,intersection_roadway_name,between_street_number_1,...,day_of_week,id,year,month,day,hour,minute,main_roadway,intersection_roadway,intersection_indicator
7,LOUISVILLE METRO POLICE DEPT,NaN,NaN,15TH,ST,S,1.928,NaN,MUHAMMAD ALI,NaN,...,WEDNESDAY,7,2010,2,24,10,0,15TH,MUHAMMAD ALI,True
8,LOUISVILLE METRO POLICE DEPT,US0031E,NaN,BARDSTOWN,RD,NaN,15.058,NaN,LONGEST,NaN,...,SATURDAY,8,2010,3,6,9,35,BARDSTOWN,LONGEST,True
9,LOUISVILLE METRO POLICE DEPT,US0150,NaN,BROADWAY,NaN,W,2.382,NaN,4TH,NaN,...,MONDAY,9,2010,3,8,21,0,BROADWAY,4TH,True
10,LOUISVILLE METRO POLICE DEPT,US0031E,NaN,BARDSTOWN,RD,NaN,14.545,NaN,SHERWOOD,NaN,...,TUESDAY,10,2010,3,9,15,50,BARDSTOWN,SHERWOOD,True
24,LOUISVILLE METRO POLICE DEPT,US0031E,NaN,BARDSTOWN,RD,NaN,14.560,NaN,SHERWOOD,NaN,...,THURSDAY,24,2010,4,15,22,0,BARDSTOWN,SHERWOOD,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,LOUISVILLE METRO POLICE DEPT,US0031E,NaN,BARDSTOWN,RD,NaN,11.346,NaN,BASHFORD MANOR,NaN,...,THURSDAY,1281,2018,4,5,21,9,BARDSTOWN,BASHFORD MANOR,True
1282,LOUISVILLE METRO POLICE DEPT,KY1065,NaN,OUTER,LOOP,NaN,6.046,KY0061,PRESTON,NaN,...,SUNDAY,1282,2018,4,29,20,50,OUTER,PRESTON,True
1307,LOUISVILLE METRO POLICE DEPT,NaN,NaN,CARDINAL,BLVD,W,0.181,KY1020,3RD,NaN,...,THURSDAY,1307,2019,10,10,14,22,CARDINAL,3RD,True
1308,LOUISVILLE METRO POLICE DEPT,US0031W,NaN,DIXIE,HWY,NaN,12.194,NaN,ST PAULS CHURCH,NaN,...,SATURDAY,1308,2019,10,19,12,36,DIXIE,ST PAULS CHURCH,True


In [21]:
S = pd.read_csv("data/raw/Jefferson_County_KY_Signalized_Intersections.csv")

S

,X,Y,OBJECTID,UNITID,SIGID,MAINSTREET,CROSSSTREET,OWNER2,ROUTE,MILEPOINT,DESCRIPTION,TIMES,OWNER,TYPE,INTID,ATMSID
0,-85.499337,38.270632,1,MPW017206,SIG017206,OLD HENRY,I 265 RAMP,S,KY-3084,1.250,OLD HENRY @ I 265 RAMP,24 HOURS,KYTC,1,76747569996,NaN
1,-85.498479,38.241589,2,MPW017211,SIG017211,SHELBYVILLE,BECKLEY WOODS,S,US 60,12.291,SHELBYVILLE @ BECKLEY WOODS,24 HOURS,KYTC,1,78959406728,NaN
2,-85.493550,38.240517,3,MPW017290,SIG017290,SHELBYVILLE,LAKE FOREST,S,US-60,12.580,SHELBYVILLE @ LAKE FOREST,24 HOURS,KYTC,1,123759407630,NaN
3,-85.570212,38.311002,4,MPW015159,SIG015159,BROWNSBORO,I 265 RAMP,S,KY-22,3.700,BROWNSBORO @ I 265 RAMP,24 HOURS,KYTC,1,143907249996,NaN
4,-85.627965,38.283916,5,MPW011726,SIG011726,HERR,LIME KILN,S,KY-22,0.440,HERR @ LIME KILN,24 HOURS,KYTC,1,184229013860,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,-85.546671,38.147768,1087,MPW000001,SIG000001,BILLTOWN RD,I 265 RAMP,S,KY-1819,5.290,BILLTOWN RD @ I 265 RAMP,24 HOURS,KYTC,1,822801889996,NaN
1087,-85.495722,38.271982,1088,MPW000002,SIG000002,OLD HENRY RD,TERRA CROSSING BLVD,S,KY-3084,1.480,OLD HENRY @ TERRA CROSSING BLVD,24 HOURS,KYTC,1,2834756E294,NaN
1088,-85.551047,38.296855,1089,MPW000003,SIG000003,WESTPORT RD,I 265 RAMP,S,KY-1447,6.470,WESTPORT RD @ I 265 RAMP,24 HOURS,KYTC,1,698671329996,NaN
1089,-85.614517,38.125808,1090,MPW000004,SIG000004,BEULAH CHURCH RD,I 265 RAMP,S,KY-864,3.450,BEULAH CHURCH RD @ I 265 RAMP,24 HOURS,KYTC,1,978905189996,NaN
